<a href="https://colab.research.google.com/github/gulabpatel/ML_Retraining_Approaches/blob/main/02_Incremental_Learning_With_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Incremental Learning- Model Retraining Approach

In [1]:
!pip install creme download

In [2]:
import pandas as pd
from download import download

path = download('https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip', '/tmp/aq', kind="zip")
messages = pd.read_csv('/tmp/aq/SMSSpamCollection', sep='\t', names=["label", "message"])
messages

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
from sklearn.model_selection import train_test_split
message_train, message_test = train_test_split(messages)

In [4]:
message_test

,label,message
2897,spam,pdate_Now - Double mins and 1000 txts on Orang...
3278,ham,Its a great day. Do have yourself a beautiful ...
3389,ham,Same as kallis dismissial in 2nd test:-).
2134,ham,Spoke with uncle john today. He strongly feels...
3997,ham,We'll you pay over like &lt;#&gt; yrs so its ...
...,...,...
554,ham,Ok. Every night take a warm bath drink a cup o...
2651,ham,Do you like shaking your booty on the dance fl...
5390,ham,Nt joking seriously i told
28,ham,"I'm back &amp; we're packing the car now, I'll..."


In [5]:
# Convert dataframe to list of tuples
messages_train = message_train.to_records(index=False)
messages_test  = message_test.to_records(index=False)

In [6]:
messages_train

rec.array([('ham', 'Ugh hopefully the asus ppl dont randomly do a reformat.'),
           ('ham', 'Got it..mail panren paru..'),
           ('ham', 'I have a rather prominent bite mark on my right cheek'),
           ...,
           ('spam', '3. You have received your mobile content. Enjoy'),
           ('ham', "Unfortunately i've just found out that we have to pick my sister up from the airport that evening so don't think i'll be going out at all. We should try to go out one of th"),
           ('ham', 'Babe ? I lost you ... Will you try rebooting ?')],
          dtype=[('label', 'O'), ('message', 'O')])

In [7]:
# Creating the pipeline
# 1st function is creating the TFIDF
# 2nd function is the naive bayes predictor
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme
from creme import metrics

model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1))
)

metric=metrics.Accuracy()
# Training the model row by row
for label, sentence in messages_train:
    model = model.fit_one(sentence, label)
    y_pred = model.predict_one(sentence)
    metric = metric.update(label, y_pred)

In [8]:
### Training Data Accuracy
metric

Accuracy: 96.10%

In [9]:
### test Data Accuracy
test_metric=metrics.Accuracy()
for label, sentence in messages_test[1:20]:
    y_pred = model.predict_one(sentence)
    test_metric = metric.update(label, y_pred)

### test Metric
test_metric

Accuracy: 96.12%

In [10]:
model.fit_one("This guy is neutral", "ham")
model.fit_one("Everybody is neutral ", "ham")

Pipeline (
  TFIDF (
    normalize=True
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of re.Pattern object at 0x7f91a5e45440>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)